In [173]:
local_llm_config = {
    "config_list": [
        {
            "model": "NotRequired",  # Loaded with LiteLLM command
            "api_key": "NotRequired",  # Not needed
            "base_url": "http://0.0.0.0:4000",  # Your LiteLLM URL
            "price": [0, 0],  # Put in price per 1K tokens [prompt, response] as free!
        }
    ],
    "cache_seed": None,  # Turns off caching, useful for testing different models
}

3. register for course  
8. check terms and conditions for a course  
11. check terms and conditions for registering course


1. taking leaves  
10. check attendaance
12. terms and conditions for taking a leave


2. fees  
4. register for mess  
5. hostel change  
6. register for hostel   
7. check terms and conditions for each feature  
9. check marksheets  


In [174]:
import shutil
import sqlite3

import pandas as pd

local_file = "student.db"
# The backup lets us restart for each tutorial section
backup_file = "student.backup.db"
shutil.copy(backup_file, local_file)

# Convert the flights to present time for our tutorial
def update_dates(file):
    shutil.copy(backup_file, file)
    conn = sqlite3.connect(file)

    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    ).name.tolist()
    tdf = {}
    for t in tables:
        tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

    for table_name, df in tdf.items():
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    del df
    del tdf
    conn.commit()
    conn.close()

    return file

db = update_dates(local_file)

In [175]:
student_id = 3

In [176]:
from typing import Annotated, Literal


def calculator(
        a: Annotated[int, "1st operand"], 
        b: Annotated[int, "2nd operand"], 
        operator: Annotated[Literal["Fall 2024", "-", "*", "/"], "operator"]
        ) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [177]:
from autogen import ConversableAgent
from autogen import register_function


# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="assistant",
    system_message="You are a helpful student support assistant. "
    " Use the provided tools to search for courses, registrations, and other information to assist the students's queries. "
    "Break down the problem into smaller parts and think step by step how to accomplish the task"
    "Return 'TERMINATE' when the task is done.",
    llm_config=local_llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

In [178]:
import sqlite3
from typing import Optional



def fetch_student_course_details() -> list[dict]:
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    query = """
    SELECT 
        c.id, c.name
    FROM 
        registered_courses rc
        JOIN courses c ON rc.course_id = c.id
        JOIN students st ON rc.student_id = st.id
    WHERE 
        st.id = ?
    """
    cursor.execute(query, (student_id,))
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results


# Register the calculator function to the two agents.
register_function(
    fetch_student_course_details,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Fetch all registered courses for the student",  # A description of the tool.
)


In [179]:
def search_courses(
    department: Optional[str] = None,
    credits: Optional[str] = None,
    semester: Annotated[Literal["Fall 2024", "Summer 2024"], "semester type"] = None,
    limit: int = 5,
) -> list[dict]:
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    query = "SELECT * FROM courses WHERE 1 = 1"
    params = []

    if department:
        query += " AND department = ?"
        params.append(department)

    if credits:
        query += " AND credits = ?"
        params.append(credits)

    if semester:
        query += " AND semester = ?"
        params.append(semester)

    query += " LIMIT ?"
    params.append(limit)

    cursor.execute(query, params)
    rows = cursor.fetchall()
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    return results

# Register the calculator function to the two agents.
register_function(
    search_courses,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Search for courses based on departments, credits, semester.",  # A description of the tool.
)


In [180]:
def create_registration(course_id: int) -> str:

    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    cursor.execute("SELECT id FROM students WHERE id = ?", (student_id,))
    existing_registration = cursor.fetchone()
    if not existing_registration:
        cursor.close()
        conn.close()
        return "No student found for the given student_id."

    cursor.execute("SELECT id FROM courses WHERE id = ?", (course_id,))
    existing_registration = cursor.fetchone()
    if not existing_registration:
        cursor.close()
        conn.close()
        return "No course found for the given course_id."

    # Check the signed-in user actually has this ticket
    cursor.execute(
        "SELECT course_id FROM registered_courses WHERE course_id = ? AND student_id = ?",
        (course_id, student_id),
    )
    current_ticket = cursor.fetchone()
    if current_ticket:
        cursor.close()
        conn.close()
        return f"Current signed-in student with ID {student_id} is already registered for {course_id}"

    cursor.execute(
        "INSERT INTO registered_courses (course_id, student_id) VALUES (?, ?)",
        (course_id, student_id)
        )
    conn.commit()

    cursor.close()
    conn.close()
    return "Course registrated successfully."

# Register the calculator function to the two agents.
register_function(
    create_registration,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Create a course registration for the student.",  # A description of the tool.
)


In [181]:
def cancel_registration(course_id: int) -> str:
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    cursor.execute("SELECT id FROM students WHERE id = ?", (student_id,))
    existing_registration = cursor.fetchone()
    if not existing_registration:
        cursor.close()
        conn.close()
        return "No student found for the given student_id."

    cursor.execute("SELECT id FROM courses WHERE id = ?", (course_id,))
    existing_registration = cursor.fetchone()
    if not existing_registration:
        cursor.close()
        conn.close()
        return "No course found for the given course_id."

    # Check the signed-in user actually has this ticket
    cursor.execute(
        "SELECT course_id FROM registered_courses WHERE course_id = ? AND student_id = ?",
        (course_id, student_id),
    )
    current_ticket = cursor.fetchone()
    if not current_ticket:
        cursor.close()
        conn.close()
        return f"Current signed-in student with ID {student_id} is not registered for {course_id}"

    cursor.execute(
        "DELETE FROM registered_courses WHERE course_id = ? AND student_id = ?", 
        (course_id, student_id)
        )
    conn.commit()

    cursor.close()
    conn.close()
    return "Course registration cancelled."

# Register the calculator function to the two agents.
register_function(
    cancel_registration,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Cancel the student's registration for a course_id",  # A description of the tool.
)


In [182]:
msg = "I want to take a new course from Computer Science department which I have not registered for before. It can be any semester. What would you recommend?"
chat_result = user_proxy.initiate_chat(
    assistant, 
    message=msg, 
    # summary_method="reflection_with_llm",
    max_turns=6)

user_proxy (to assistant):

I want to take a new course from Computer Science department which I have not registered for before. It can be any semester. What would you recommend?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_6235bdf1-95c9-4194-aa6a-bc87be86c72f): search_courses *****
Arguments: 
{"department": "Computer Science", "credits": null, "semester": null, "limit": 5}
*******************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_courses...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_6235bdf1-95c9-4194-aa6a-bc87be86c72f) *****
[{"id": 1, "name": "Introduction to Computer Science", "department": "Computer Science", "semester": "Fall 2024", "credits": 3}, {"id":

InternalServerError: Error code: 500 - {'error': {'message': 'litellm.APIConnectionError: \'arguments\'\nTraceback (most recent call last):\n  File "/Users/kaustuv/Desktop/autogen/.venv/lib/python3.12/site-packages/litellm/main.py", line 427, in acompletion\n    response = await init_response\n               ^^^^^^^^^^^^^^^^^^^\n  File "/Users/kaustuv/Desktop/autogen/.venv/lib/python3.12/site-packages/litellm/llms/ollama.py", line 495, in ollama_acompletion\n    raise e  # don\'t use verbose_logger.exception, if exception is raised\n    ^^^^^^^\n  File "/Users/kaustuv/Desktop/autogen/.venv/lib/python3.12/site-packages/litellm/llms/ollama.py", line 467, in ollama_acompletion\n    "arguments": json.dumps(function_call["arguments"]),\n                            ~~~~~~~~~~~~~^^^^^^^^^^^^^\nKeyError: \'arguments\'\n', 'type': None, 'param': None, 'code': '500'}}

In [171]:
"I have a opening in my schedule on Wednesday 10:30 to 11:30, which courses can I take?"

'I have a opening in my schedule on Wednesday 10:30 to 11:30, which courses can I take?'

In [172]:
"Are my credits enough"

'Are my credits enough'

implement safe workflows - database writing operations should be done by user permission.